# 0. Desafio

Neste projeto, utilizei a programação linear em Python com a biblioteca OR-Tools para resolver um problema de **Otimização de Campanhas de Marketing Digital**. O seguinte desafio e os dados ficcionais foram elaborados com auxílio de inteligência artificial.

A empresa fictícia em questão é um e-commerce que busca maximizar o retorno sobre investimento (ROI) de suas campanhas de marketing digital. O orçamento disponível é de $50.000, que precisa ser distribuído entre três principais canais de marketing: Google Ads, Facebook Ads e E-mail Marketing. Cada canal tem seu próprio custo por conversão e ROI estimado, além de limites mínimos e máximos de investimento que precisam ser respeitados.

O desafio consiste em criar um modelo de programação linear que permita a alocação eficiente do orçamento entre os canais, visando o máximo retorno. Com base nos dados fornecidos, o modelo precisa determinar quanto deve ser investido em cada canal para otimizar os resultados da campanha.


# 1. Carregando Bibliotecas

In [49]:
import pandas as pd # importa a biblioteca pandas
from ortools.linear_solver import pywraplp  # importa o pywraplp da biblioteca ortools
from tabulate import tabulate # Importa o tabulate

# 2. Visualizando dados

In [48]:
dados_mkt = pd.read_csv('Desafio_de_negócio_com_programacao_linear.csv') # importa o arquivo csv
display(dados_mkt)

,canal_marketing,custo _por_conversao,lim_min_investimento,lim_max_investimento,roi_por_dolar_investido
0,Google Ads,15,10000,25000,"2,5"
1,Facebook Ads,20,5000,15000,3
2,E-mail Marketing,10,2000,10000,"1,8"


## Dicionário de Dados:

- **canal_marketing**: Lista dos canais de marketing utilizados.
- **custo_por_conversao**: Custo por conversao, ou seja, o valor necessário para cada conversão em um canal.
- **lim_min_investimento**: O investimento mínimo exigido para cada canal.
- **lim_max_investimento**: O investimento máximo permitido em cada canal.
- **roi_por_dolar_investido (retorno esperado)**: Retorno sobre o investimento por dólar investido em cada canal.

# 2. Resolução

In [52]:
solver = pywraplp.Solver.CreateSolver('SCIP')  # Cria um solver para programação inteira mista

In [53]:
lim_min_facebook = 5000	  # Mínimo de investimento Facebook Ads
lim_max_facebook = 15000  # Máximo de investimento Facebook Ads

lim_min_google = 10000   # Mínimo de investimento Google Ads
lim_max_google = 25000    # Máximo de investimento Google Ads

lim_min_email = 2000      # Mínimo de investimento Email
lim_max_email = 10000     # Máximo de investimento Email


custo_conv_fb = 20         # Custo de conversão Facebook Ads
custo_conv_google = 15     # Custo de conversão Google Ads
custo_conv_email = 10      # Custo de conversão Email


# ROI por dólar investido
roi_facebook = 3
roi_google = 2.5
roi_email = 1.8

# Define variáveis de conversão
facebook_investimento = solver.NumVar(lim_min_facebook, lim_max_facebook, 'Facebook Ads')
google_investimento = solver.NumVar(lim_min_google, lim_max_google, 'Google Ads')
email_investimento = solver.NumVar(lim_min_email, lim_max_email, 'Email Marketing')
# Definimos as variáveis como NumVar pois tratam-se de valores de investimento
# e obrigamos que respeitem os limites previammente



orcamento_total = 50000 # Define nosso limite de gasto

# Define nosso orçamento total como restrição
solver.Add(facebook_investimento + google_investimento + email_investimento  <= orcamento_total)

# Calculo do ROI total
total_roi = (
    (facebook_investimento / custo_conv_fb) * roi_facebook +
    (google_investimento / custo_conv_google) * roi_google +
    (email_investimento / custo_conv_email) * roi_email
)

# Define a maximização do ROI total
solver.Maximize(total_roi)

# Resolve o problema com o solver
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    # Calculo do ROI total
    roi_total = (
        (facebook_investimento.solution_value() / custo_conv_fb) * roi_facebook +
        (google_investimento.solution_value() / custo_conv_google) * roi_google +
        (email_investimento.solution_value() / custo_conv_email) * roi_email
    )

    # Estrutura os dados para exibição
    data = [
        ['Facebook Ads', '${:.2f}'.format(facebook_investimento.solution_value())],
        ['Google Ads', '${:.2f}'.format(google_investimento.solution_value())],
        ['Email Marketing', '${:.2f}'.format(email_investimento.solution_value())],
        ['ROI Total', '${:.2f}'.format(roi_total)]
    ]

    print(tabulate(data, tablefmt='rounded_grid'))

else:

    erro = [
        ['O problema não apresenta solução viável']  # Caso contrário, exiba uma mensagem de erro
    ]
    print(tabulate(erro, tablefmt='rounded_grid'))

╭─────────────────┬───────────╮
│ Facebook Ads    │ $15000.00 │
├─────────────────┼───────────┤
│ Google Ads      │ $25000.00 │
├─────────────────┼───────────┤
│ Email Marketing │ $10000.00 │
├─────────────────┼───────────┤
│ ROI Total       │ $8216.67  │
╰─────────────────┴───────────╯


A solução encontrada pelo modelo de otimização retorna os valores ideais de investimento em cada canal, maximizando o ROI total para o orçamento fornecido. Além disso, o código gera uma tabela formatada que exibe de forma objetiva os investimentos sugeridos em cada canal de marketing, bem como o ROI total estimado.